# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2: Web Scraping with Beautiful Soup

**Harvard University**<br>
**Fall 2019**<br>
**Instructors:** Pavlos Protopapas, Kevin Rader, and Chris Tanner <br>
**Lab Instructors:** Chris Tanner and Eleni Kaxiras<br>

**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, and Eleni Kaxiras

---

In [11]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn.apionly as sns
import time

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Introduction to Web Servers and HTTP </li>
<li> Download webpages and get basic properties </li>
<li> Parse the page with Beautiful Soup</li>
<li> String formatting</li>
<li> Additonal Python/Homework Comment</li>
<li> Walkthrough Example</li>
</ol>

# Learning Goals

- Understand the structure of a web page
- Understand how to use Beautiful soup to scrape content from web pages.
- Feel comfortable storing and manipulating the content in various formats.
- Understand how to convert structured format into a Pandas DataFrame

In this lab, we'll scrape Goodread's Best Books list:

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 .

We'll walk through scraping the list pages for the book names/urls. First, we start with an even simpler example.

*This lab corresponds to lectures #2 and #3 and maps on to Homework #1 and further.*

# 1. Introduction to Web Servers and HTTP

A web server is just a computer -- usually a powerful one, but ultimately it's just another computer -- that runs a long/continuous process that listens for requests on a pre-specified (Internet) _port_ on your computer. It responds to those requests via a protocol called HTTP (HyperText Transfer Protocol). HTTPS is the secure version. When we use a web browser and navigate to a web page, our browser is actually sending a request on our behalf to a specific web server. The browser request is essentially saying "hey, please give me the web page contents", and it's up to the browser to correctly render that raw content into a coherent manner, dependent on the format of the file. For example, HTML is one format, XML is another format, and so on.

Ideally (and usually), the web server complies with the request and all is fine. As part of this communication exchange with web servers, the server also sends a status code.
- If the code starts with a **2**, it means the request was successful.
- If the code starts with a **4**, it means there was a client error (you, as the user, are the client). For example, ever receive a 404 File Not Found error because a web page doesn't exist? This is an example of a client error, because you are requesting a bogus item.
- If the code starts with a **5**, it means there was a server error (often that your request was incorrectly formed).

[Click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

As an analogy, you can think of a web server as being like a server at a restaurant; its goal is _serve_ you your requests. When you try to order something not on the menu (i.e., ask for a web page at a wrong location), the server says 'sorry, we don't have that' (i.e., 404, client error; your mistake).

**IMPORTANT:**
As humans, we visit pages in a sane, reasonable rate. However, as we start to scrape web pages with our computers, we will be sending requests with our code, and thus, we can make requests at an incredible rate. This is potentially dangerous because it's akin to going to a restaurant and bombarding the server(s) with thousands of food orders. Very often, the restaurant will ban you (i.e., Harvard's network gets banned from the website, and you are potentially held responsible in some capacity?). It is imperative to be responsible and careful. In fact, this act of flooding web pages with requests is the single-most popular, yet archiac, method for maliciously attacking websites / computers with Internet connections. In short, be respectful and careful with your decisions and code. It is better to err on the side of caution, which includes using the **``time.sleep()`` function** to pause your code's execution between subsequent requests. ``time.sleep(2)`` should be fine when making just a few dozen requests. Each site has its own rules, which are often visible via their site's ``robots.txt`` file.

### Additional Resources

**HTML:** if you are not familiar with HTML see https://www.w3schools.com/html/ or one of the many tutorials on the internet.

**Document Object Model (DOM):** for more on this programming interface for HTML and XML documents see https://www.w3schools.com/js/js_htmldom.asp.

# 2. Download webpages and get basic properties

``Requests`` is a highly useful Python library that allows us to fetch web pages.
``BeautifulSoup`` is a phenomenal Python library that allows us to easily parse web content and perform basic extraction.

If one wishes to scrape webpages, one usually uses ``requests`` to fetch the page and ``BeautifulSoup`` to parse the page's meaningful components. Webpages can be messy, despite having a structured format, which is why BeautifulSoup is so handy.

Let's get started:

In [3]:
from bs4 import BeautifulSoup
import requests

To fetch a webpage's content, we can simply use the ``get()`` function within the requests library:

In [4]:
url = "https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url) # you can use any URL that you wish

The response variable has many highly useful attributes, such as:
- status_code
- text
- content

Let's try each of them!

### response.status_code

In [5]:
response.status_code

200

You should have received a status code of 200, which means the page was successfully found on the server and sent to receiver (aka client/user/you). [Again, you can click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

### response.text


In [13]:
response.text

'<!doctype html><html class="no-js" lang="en"><head><script>\n   try {var _sf_startpt=(new Date()).getTime();} catch(e){}\n</script>\n<meta itemscope itemtype="https://schema.org/SpeakableSpecification" itemref="speakable" /><title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title><meta id="speakable" itemprop="cssSelector" content="[data-is-speakable]" /><meta id="google-site-verification" name="google-site-verification" content="1VqzbDm4ukeFVHOoq5LjIKA2fVKZD-EYv8cXZhKXSfU" /><meta charset="utf-8" /><meta name="robots" content="noarchive,index,follow" /><meta name="Rating" content="General" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><meta name="npr-pjax" /><meta name="date" content="2018-11-05" /><link rel="canonical" href="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect" /><meta name="description" content="The polls 

Holy moly! That looks awful. If we use our browser to visit the URL, then right-click the page and click 'View Page Source', we see that it is identical to this chunk of glorious text.

### response.content

In [14]:
response.content

b'<!doctype html><html class="no-js" lang="en"><head><script>\n   try {var _sf_startpt=(new Date()).getTime();} catch(e){}\n</script>\n<meta itemscope itemtype="https://schema.org/SpeakableSpecification" itemref="speakable" /><title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title><meta id="speakable" itemprop="cssSelector" content="[data-is-speakable]" /><meta id="google-site-verification" name="google-site-verification" content="1VqzbDm4ukeFVHOoq5LjIKA2fVKZD-EYv8cXZhKXSfU" /><meta charset="utf-8" /><meta name="robots" content="noarchive,index,follow" /><meta name="Rating" content="General" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><meta name="npr-pjax" /><meta name="date" content="2018-11-05" /><link rel="canonical" href="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect" /><meta name="description" content="The polls

What?! This seems identical to the ``.text`` field. However, the careful eye would notice that the very 1st characters differ; that is, ``.content`` has a *b'* character at the beginning, which in Python syntax denotes that the data type is bytes, whereas the ``.text`` field did not have it and is a regular String.

Ok, so that's great, but how do we make sense of this text? We could manually parse it, but that's tedious and difficult. As mentioned, BeautifulSoup is specifically designed to parse this exact content (any webpage content).

## BEAUTIFUL SOUP
![title](images/soup_for_you.jpg) (property of NBC)


The [documentation for BeautifulSoup is found here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

A BeautifulSoup object can be initialized with the ``.content`` from request and a flag denoting the type of parser that we should use. For example, we could specify ``html.parser``, ``lxml``, etc [documentation here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#differences-between-parsers). Since we are interested in standard webpages that use HTML, let's specify the html.parser:

In [15]:
soup = BeautifulSoup(response.content, "html.parser")
soup

<!DOCTYPE doctype html>
<html class="no-js" lang="en"><head><script>
   try {var _sf_startpt=(new Date()).getTime();} catch(e){}
</script>
<meta itemref="speakable" itemscope="" itemtype="https://schema.org/SpeakableSpecification"/><title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title><meta content="[data-is-speakable]" id="speakable" itemprop="cssSelector"/><meta content="1VqzbDm4ukeFVHOoq5LjIKA2fVKZD-EYv8cXZhKXSfU" id="google-site-verification" name="google-site-verification"/><meta charset="utf-8"/><meta content="noarchive,index,follow" name="robots"/><meta content="General" name="Rating"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta name="npr-pjax"/><meta content="2018-11-05" name="date"/><link href="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect" rel="canonical"/><meta content="The polls show a Democratic adv

Alright! That looks a little better; there's some whitespace formatting, adding some structure to our content! HTML code is structured by `<tags>`. Every tag has an opening and closing portion, denoted by ``< >`` and ``</ >``, respectively. If we want just the text (not the tags), we can use:

In [48]:
soup.get_text()

'\n   try {var _sf_startpt=(new Date()).getTime();} catch(e){}\n\nWhat If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR\n   window.NPR = window.NPR || {};\nNPR.ServerConstants = {"cbHost":"npr.org","webHost":"https:\\/\\/www.npr.org","embedHost":"https:\\/\\/www.npr.org","webHostSecure":"https:\\/\\/secure.npr.org","apiHost":"https:\\/\\/api.npr.org","serverMediaCache":"https:\\/\\/media.npr.org","googleAnalyticsAccount":"UA-5828686-4","nielsenSFCode":"dcr","nielsenAPN":"NPR-dcr","shouldShowHPLocalContent":true,"readingServiceHostname":"https:\\/\\/reading.api.npr.org"};\nNPR.serverVars = {"storyId":"664395755","facebookAppId":"138837436154588","webpackPublicPath":"https:\\/\\/s.npr.org\\/templates\\/javascript\\/dist\\/bundles\\/","persistenceVersion":"e2193dbd58d7e71fdaffbd399767e8dc","isBuildOut":true,"topicIds":["P139482413","1001","1002","1003","1014","1059"],"primaryTopic":"Elections","topics":["Elections","News","Home Page Top Stories","Nati

There's some tricky Javascript still nesting within it, but it definitely cleaned up a bit. On other websites, you may find even clearer text extraction.

As detailed in the [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), the easiest way to navigate through the tags is to simply name the tag you're interested in. For example:

In [27]:
soup.head # fetches the head tag, which ecompasses the title tag

<head><script>
   try {var _sf_startpt=(new Date()).getTime();} catch(e){}
</script>
<meta itemref="speakable" itemscope="" itemtype="https://schema.org/SpeakableSpecification"/><title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title><meta content="[data-is-speakable]" id="speakable" itemprop="cssSelector"/><meta content="1VqzbDm4ukeFVHOoq5LjIKA2fVKZD-EYv8cXZhKXSfU" id="google-site-verification" name="google-site-verification"/><meta charset="utf-8"/><meta content="noarchive,index,follow" name="robots"/><meta content="General" name="Rating"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta name="npr-pjax"/><meta content="2018-11-05" name="date"/><link href="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect" rel="canonical"/><meta content="The polls show a Democratic advantage in the House and a Republican one in the Senate

Usually head tags are small and only contain the most important contents; however, here, there's some Javascript code. The ``title`` tag resides within the head tag.

In [29]:
soup.title # we can specifically call for the title tag

<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>

This result includes the tag itself. To get just the text within the tags, we can use the ``.name`` property.

In [17]:
soup.title.string

'What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR'

We can navigate to the parent tag (the tag that encompasses the current tag) via the ``.parent`` attribute:

In [32]:
soup.title.parent.name

'head'

# 3. Parse the page with Beautiful Soup
In HTML code, paragraphs are often denoated with a ``<p>`` tag.

In [34]:
soup.p

<p class="byline__name byline__name--block">
<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>
</p>

This returns the first paragraph, and we can access properties of the given tag with the same syntax we use for dictionaries and dataframes:

In [36]:
soup.p['class']

['byline__name', 'byline__name--block']

In addition to 'paragraph' (aka p) tags, link tags are also very common and are denoted by ``<a>`` tags

In [39]:
soup.a

<a class="skiplink" href="#mainContent">Skip to main content</a>

It is called the a tag because links are also called 'anchors'. Nearly every page has multiple paragraphs and anchors, so how do we access the subsequent tags? There are two common functions, `.find()` and `.find_all()`.

In [45]:
soup.find('title')

<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>

In [46]:
soup.find_all('title')

[<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>]

Here, the results were seemingly the same, since there is only one title to a webpage. However, you'll notice that ``.find_all()`` returned a list, not a single item. Sure, there was only one item in the list, but it returned a list. As the name implies, find_all() returns all items that match the passed-in tag.

In [47]:
soup.find_all('a')

[<a class="skiplink" href="#mainContent">Skip to main content</a>,
 <a class="skiplink" href="https://help.npr.org/customer/en/portal/articles/2535181-what-are-the-keyboard-shortcuts-for-using-the-npr-org-audio-player-">Keyboard shortcuts for audio player</a>,
 <a data-metrics-action="click npr logo" href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>,
 <a data-metrics-action="click npr shop" href="https://shop.npr.org">NPR Shop</a>,
 <a data-metrics-action="click npr logo" href="/">Home</a>,
 <a data-metrics-action="click news" href="/sections/news/">News</a>,
 <a data-metrics-action="click national" href="/sections/national/">National</a>,
 <a data-metrics-action="click world" href="/sections/world/">World</a>,
 <a data-metrics-action="click politics" href="/sections/politics/">Politics</a>,
 <a data-metrics-action="click business" href="/sections/business/">Business</a>,
 <a data-metrics-action="click health" href="/sections/health/">Health</a>

Look at all of those links! Amazing. It might be hard to read but the **href** portion of an *a* tag denotes the URL, and we can capture it via the ``.get()`` function.

In [27]:
for link in soup.find_all('a'): # we could optionally pass the href=True flag .find_all('a', href=True)
    print(link.get('href'))

#mainContent
https://help.npr.org/customer/en/portal/articles/2535181-what-are-the-keyboard-shortcuts-for-using-the-npr-org-audio-player-
/
https://shop.npr.org
/
/sections/news/
/sections/national/
/sections/world/
/sections/politics/
/sections/business/
/sections/health/
/sections/science/
/sections/technology/
/sections/codeswitch/
/sections/arts/
/books/
/sections/movies/
/sections/television/
/sections/pop-culture/
/sections/food/
/sections/art-design/
/sections/performing-arts/
/music/
https://www.npr.org/series/tiny-desk-concerts/
https://www.npr.org/sections/allsongs/
https://www.npr.org/series/538009815/turning-the-tables
https://www.npr.org/sections/music-news/
https://www.npr.org/sections/new-music/
https://www.npr.org/series/689345495/best-music-of-2019
/programs/
/programs/morning-edition/
/programs/weekend-edition-saturday/
/programs/weekend-edition-sunday/
/programs/all-things-considered/
/programs/fresh-air/
/podcasts/510318/up-first
https://www.npr.org/podcasts/510313/

Many of those links are relative to the current URL (e.g., /section/news/).

In [28]:
paragraphs = soup.find_all('p')
paragraphs

[<p class="byline__name byline__name--block">
 <a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
       Domenico Montanaro
     </a>
 </p>, <p>
                 Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
                 
                 <b aria-label="Image credit" class="credit">
                     
                     Scott Olson/Getty Images
                     
                 </b>
 <b class="hide-caption"><b>hide caption</b></b>
 </p>, <p class="caption">Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.</p>, <p>There's a lot that can happen Tuesday, the culmination of a long midterm election campaign that will provide the first nationwide measure of the U.S. electorate since Donald Trump was elected president.</p>, <p>One narrat

If we want just the paragraph text:

In [30]:
for pa in paragraphs:
    print(pa.get_text())



      Domenico Montanaro
    


                Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
                
                
                    
                    Scott Olson/Getty Images
                    
                
hide caption

Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
There's a lot that can happen Tuesday, the culmination of a long midterm election campaign that will provide the first nationwide measure of the U.S. electorate since Donald Trump was elected president.
One narrative has become dominant: that Democrats are likely to gain control of the House and Republicans hold the Senate, if not expand their majority there. That narrative is based largely on national polls, and caution should be urged. Pollsters have made a lot of adjustments to hopefully correct what they got wrong in 2016, but they can't tell y

Since there are multiple tags and various attributes, it is useful to check the data type of BeautifulSoup objects:

In [31]:
type(soup.find('p'))

bs4.element.Tag

Since the ``.find()`` function returns a BeautifulSoup element, we can tack on multiple calls that continue to return elements:

In [68]:
soup.find('p')

<p class="byline__name byline__name--block">
<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>
</p>

In [69]:
soup.find('p').find('a')

<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>

In [70]:
soup.find('p').find('a').attrs['href'] # att

'https://www.npr.org/people/392602474/domenico-montanaro'

In [71]:
soup.find('p').find('a').text

'\n      Domenico Montanaro\n    '

That doesn't look pretty, but it makes sense because if you look at what ``.find('a')`` returned, there is plenty of whitespace. We can remove that with Python's built-in ``.strip()`` function.

In [72]:
soup.find('p').find('a').text.strip()

'Domenico Montanaro'

**NOTE:** above, we accessed the attributes of a link by using the property ``.attrs``. ``.attrs`` takes a dictionary as a parameter, and in the example above, we only provided the _key_, not a _value_, too. That is, we only cared that the ``<a>`` tag had an attribute named ``href`` (which we grabbed by typing that command), and we made no specific demands on what the value must be. In other words, regardless of the value of _href_, we grabbed that element. Alternatively, if you inspect your HTML code and notice select regions for which you'd like to extract text, you can specify it as part of the attributes, too!

For example, in the full ``response.text``, we see the following line:

``<header class="npr-header" id="globalheader" aria-label="NPR header">``

Let's say that we know that the information we care about is within tags that match this template (i.e., **class** is an attribute, and its value is **'npr-header'**).

In [36]:
soup.find('header', attrs={'class':'npr-header'})

<header aria-label="NPR header" class="npr-header" id="globalheader"><nav aria-label="main navigation" class="navigation">
<div class="navigation__top-level-wrap" data-metrics-all="" data-metrics-category="global navigation">
<ul class="navigation__top-level">
<li class="navigation__home">
<a data-metrics-action="click npr logo" href="/">
<img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
</a>
</li>
<li id="navigation__station-mount"></li>
<li class="navigation__account" id="user-account-dropdown-mount"></li>
<li class="navigation__shop">
<a data-metrics-action="click npr shop" href="https://shop.npr.org">NPR Shop</a>
</li>
<li id="navigation__station-donate-mount"></li>
<li class="navigation__toggle-wrap">
<button aria-expanded="false" aria-haspopup="true" class="navigation__toggle navigation__toggle--open" data-metrics-action="open navigation menu" id="navigation__toggle--open">
                    Open Navigation Menu
                </button>
</li>
</ul>
</di

This matched it! We could then continue further processing by tacking on other commands:

In [77]:
soup.find('header', attrs={'class':'npr-header'}).find_all("li") # li stands for list items

[<li class="navigation__home">
 <a data-metrics-action="click npr logo" href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>
 </li>,
 <li id="navigation__station-mount"></li>,
 <li class="navigation__account" id="user-account-dropdown-mount"></li>,
 <li class="navigation__shop">
 <a data-metrics-action="click npr shop" href="https://shop.npr.org">NPR Shop</a>
 </li>,
 <li id="navigation__station-donate-mount"></li>,
 <li class="navigation__toggle-wrap">
 <button aria-expanded="false" aria-haspopup="true" class="navigation__toggle navigation__toggle--open" data-metrics-action="open navigation menu" id="navigation__toggle--open">
                     Open Navigation Menu
                 </button>
 </li>,
 <li class="menu__item menu__item--home">
 <div class="menu__item-inner"><a data-metrics-action="click npr logo" href="/">Home</a></div>
 </li>,
 <li class="menu__item menu__item--news menu__item--has-submenu" data-metrics-hover="toggle news drawer"

This returns all of our list items, and since it's within a particular header section of the page, it appears they are links to menu items for navigating the webpage. If we wanted to grab just the links within these:

In [82]:
menu_links = set()
for list_item in soup.find('header', attrs={'class':'npr-header'}).find_all("li"):
    for link in list_item.find_all('a', href=True):
        menu_links.add(link)
menu_links # a unique set of all the seemingly important links in the header

{<a data-metrics-action="click about npr" href="/about/">About NPR</a>,
 <a data-metrics-action="click all programs" href="/programs/">More Shows &amp; Podcasts</a>,
 <a data-metrics-action="click all songs considered" href="https://www.npr.org/sections/allsongs/">
         All Songs Considered
     </a>,
 <a data-metrics-action="click all things considered" href="/programs/all-things-considered/">
 <img alt="" src="https://media.npr.org/chrome/programs/logos/all-things-considered.png"/>
                                     All Things Considered
                                 </a>,
 <a data-metrics-action="click art &amp; design" href="/sections/art-design/">Art &amp; Design</a>,
 <a data-metrics-action="click arts &amp; life" href="/sections/arts/">Arts &amp; Life</a>,
 <a data-metrics-action="click best music of 2019" href="https://www.npr.org/series/689345495/best-music-of-2019">
         Best Music Of 2019
     </a>,
 <a data-metrics-action="click books" href="/books/">Books</a>,

## TAKEAWAY LESSON
The above tutorial isn't meant to be a study guide to memorize; its point is to show you the most important functionaity that exist within BeautifulSoup, and to illustrate how one can access different pieces of content. No two web scraping tasks are identical, so it's useful to play around with code and try different things, while using the above as examples of how you may navigate between different tags and properties of a page. Don't worry; we are always here to help when you get stuck!

# String formatting
As we parse webpages, we may often want to further adjust and format the text to a certain way.

For example, say we wanted to scrape a polical website that lists all US Senators' name and office phone number. We may want to store information for each senator in a dictionary. All senators' information may be stored in a list. Thus, we'd have a list of dictionaries. Below, we will initialize such a list of dictionary (it has only 3 senators, for illustrative purposes, but imagine it contains many more).

In [2]:
# this is a bit clumsy of an initialization, but we spell it out this way for clarity purposes
# NOTE: imagine the dictionary were constructed in a more organic manner
senator1 = {"name":"Lamar Alexander", "number":"555-229-2812"}
senator2 = {"name":"Tammy Baldwin", "number":"555-922-8393"}
senator3 = {"name":"John Barrasso", "number":"555-827-2281"}
senators = [senator1, senator2, senator3]
print(senators)

[{'name': 'Lamar Alexander', 'number': '555-229-2812'}, {'name': 'Tammy Baldwin', 'number': '555-922-8393'}, {'name': 'John Barrasso', 'number': '555-827-2281'}]


In the real-world, we may not want the final form of our information to be in a Python dictionary; rather, we may need to send an email to people in our mailing list, urging them to call their senators. If we have a templated format in mind, we can do the following:

In [3]:
email_template = """Please call {name} at {number}"""
for senator in senators:
    print(email_template.format(**senator))

Please call Lamar Alexander at 555-229-2812
Please call Tammy Baldwin at 555-922-8393
Please call John Barrasso at 555-827-2281


**Please [visit here](https://docs.python.org/3/library/stdtypes.html#str.format)** for further documentation
                      
Alternatively, one can also format their text via the ``f'-strings`` property. [See documentation here](https://docs.python.org/3/reference/lexical_analysis.html#f-strings). For example, using the above data structure and goal, one could yield identical results via:

In [11]:
for senator in senators:
    print(f"Please call {senator['name']} at {senator['number']}")

Please call Lamar Alexander at 555-229-2812
Please call Tammy Baldwin at 555-922-8393
Please call John Barrasso at 555-827-2281


Additionally, sometimes we wish to search large strings of text. If we wish to find all occurrences within a given string, a very mechanical, procedural way of doing it would be to use the ``.find()`` function in Python and to repeatedly update the starting index from which we are looking.

## Regular Expressions
A way more suitable and powerful way is to use Regular Expressions, which is a pattern matching mechanism used throughout Computer Science and programming (it's not just specific to Python). A tutorial on Regular Expressions (aka regex) is beond this lab, but below are many great resources that we recommend, if you are interested in them (could be very useful for a homework problem):
- https://docs.python.org/3.3/library/re.html
- https://regexone.com
- https://docs.python.org/3/howto/regex.html.

# Additonal Python/Homework Comment
In Homework #1, we ask you to complete functions that have signatures with a syntax you may not have seen before:

``def create_star_table(starlist: list) -> list:``

To be clear, this syntax merely means that the input parameter must be a list, and the output must be a list. It's no different than any other function, it just puts a requirement on the behavior of the function.

It is **typing** our function. Please [see this documention if you have more questions.](https://docs.python.org/3/library/typing.html)

# Walkthrough Example (of Web Scraping)
We're going to see the structure of Goodread's best books list (**NOTE: Goodreads is described a little more within the other Lab2_More_Pandas.ipynb notebook)**. We'll use the Developer tools in chrome, safari and firefox have similar tools available. To get this page we use the `requests` module. But first we should check if the company's policy allows scraping. Check the [robots.txt](https://www.goodreads.com/robots.txt) to find what sites/elements are not accessible. Please read and verify.

![](images/goodreads1.png)

In [23]:
url="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url)
# response.status_code
# response.content

# Beautiful Soup (library) time!
soup = BeautifulSoup(response.content, "html.parser")
    #print(soup)
    # soup.prettify()
soup.find("title")

    # Q1: how do we get the title's text?

    # Q2: how do we get the webpage's entire content?

IndentationError: unexpected indent (<ipython-input-23-630b934c476c>, line 2)

In [6]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
print(url)
page = requests.get(url)

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1


We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found , ok. (See lecture notes.)

In [7]:
page.status_code # 200 is good

200

In [8]:
page.text[:5000]

'<!DOCTYPE html>\n<html class="desktop\n">\n<head>\n  <title>Best Books Ever (56897 books)</title>\n\n<meta content=\'55,397 books based on 191605 votes: The Hunger Games by Suzanne Collins, Harry Potter and the Order of the Phoenix by J.K. Rowling, To Kill a Mockingbird...\' name=\'description\'>\n<meta content=\'telephone=no\' name=\'format-detection\'>\n<link href=\'https://www.goodreads.com/list/show/1.Best_Books_Ever\' rel=\'canonical\'>\n\n\n\n    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();\n </script>\n  <script type="text/javascript">\n    var ue_mid = "A1PQBFHBHS6YH1";\n    var ue_sn = "www.goodreads.com";\n    var ue_furl = "fls-na.amazon.com";\n    var ue_sid = "865-1115738-3157714";\n    var ue_id = "KYNGZWAZ0H7376V1SV7J";\n\n    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}functio

Let us write a loop to fetch 2 pages of "best-books" from goodreads. Notice the use of a format string. This is an example of old-style python format strings

In [9]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
for i in range(1,3):
    bookpage=str(i)
    stuff=requests.get(URLSTART+BESTBOOKS+bookpage)
    filetowrite="files/page"+ '%02d' % i + ".html"
    print("FTW", filetowrite)
    fd=open(filetowrite,"w")
    fd.write(stuff.text)
    fd.close()
    time.sleep(2)

FTW files/page01.html
FTW files/page02.html


## 2. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [11]:
bookdict={}
for i in range(1,3):
    books=[]
    stri = '%02d' % i
    filetoread="files/page"+ stri + '.html'
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, 'html.parser')
    for e in soup.select('.bookTitle'):
        books.append(e['href'])
    print(books[:10])
    bookdict[stri]=books
    fd=open("files/list"+stri+".txt","w")
    fd.write("\n".join(books))
    fd.close()

FTW files/page01.html
['/book/show/2767052-the-hunger-games', '/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix', '/book/show/2657.To_Kill_a_Mockingbird', '/book/show/1885.Pride_and_Prejudice', '/book/show/41865.Twilight', '/book/show/19063.The_Book_Thief', '/book/show/7613.Animal_Farm', '/book/show/11127.The_Chronicles_of_Narnia', '/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set', '/book/show/18405.Gone_with_the_Wind']
FTW files/page02.html
['/book/show/43763.Interview_with_the_Vampire', '/book/show/153747.Moby_Dick_or_the_Whale', '/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban', '/book/show/4989.The_Red_Tent', '/book/show/37435.The_Secret_Life_of_Bees', '/book/show/7171637-clockwork-angel', '/book/show/6.Harry_Potter_and_the_Goblet_of_Fire', '/book/show/2187.Middlesex', '/book/show/49552.The_Stranger', '/book/show/16299.And_Then_There_Were_None']


Here is George Orwell's 1984

In [12]:
bookdict['02'][0]

'/book/show/43763.Interview_with_the_Vampire'

 Lets go look at the first URLs on both pages

![](images/goodreads2.png)

## 3. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [13]:
furl=URLSTART+bookdict['02'][0]
furl

'https://www.goodreads.com/book/show/43763.Interview_with_the_Vampire'

![](images/goodreads3.png)

In [14]:
fstuff=requests.get(furl)
print(fstuff.status_code)

200


In [15]:
#d=BeautifulSoup(fstuff.text, 'html.parser')
# try this to take care of arabic strings
d = BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")

In [16]:
d.select("meta[property='og:title']")[0]['content']

'Interview with the Vampire (The Vampire Chronicles, #1)'

Lets get everything we want...

In [19]:
#d=BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")
print(
"title", d.select_one("meta[property='og:title']")['content'],"\n",
"isbn", d.select("meta[property='books:isbn']")[0]['content'],"\n",
"type", d.select("meta[property='og:type']")[0]['content'],"\n",
"author", d.select("meta[property='books:author']")[0]['content'],"\n",
#"average rating", d.select_one("span.average").text,"\n",
"ratingCount", d.select("meta[itemprop='ratingCount']")[0]["content"],"\n"
#"reviewCount", d.select_one("span.count")["title"]
)

title Interview with the Vampire (The Vampire Chronicles, #1) 
 isbn 9780345476876 
 type books.book 
 author https://www.goodreads.com/author/show/7577.Anne_Rice 
 ratingCount 446635 



Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [20]:
"list{:0>2}.txt".format(3)

'list03.txt'

In [21]:
a = "4"
b = 4
class Four:
    def __str__(self):
        return "Fourteen"
c=Four()

In [22]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

'The hazy cat jumped over the 4 and 4 and Fourteen'

## 4. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [23]:
fetched=[]
for i in range(1,3):
    with open("files/list{:0>2}.txt".format(i)) as fd:
        counter=0
        for bookurl_line in fd:
            if counter > 4:
                break
            bookurl=bookurl_line.strip()
            stuff=requests.get(URLSTART+bookurl)
            filetowrite=bookurl.split('/')[-1]
            filetowrite="files/"+str(i)+"_"+filetowrite+".html"
            print("FTW", filetowrite)
            fd=open(filetowrite,"w")
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite)
            time.sleep(2)
            counter=counter+1
            
print(fetched)

FTW files/1_2767052-the-hunger-games.html
FTW files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
FTW files/1_2657.To_Kill_a_Mockingbird.html
FTW files/1_1885.Pride_and_Prejudice.html
FTW files/1_41865.Twilight.html
FTW files/2_43763.Interview_with_the_Vampire.html
FTW files/2_153747.Moby_Dick_or_the_Whale.html
FTW files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
FTW files/2_4989.The_Red_Tent.html
FTW files/2_37435.The_Secret_Life_of_Bees.html
['files/1_2767052-the-hunger-games.html', 'files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html', 'files/1_2657.To_Kill_a_Mockingbird.html', 'files/1_1885.Pride_and_Prejudice.html', 'files/1_41865.Twilight.html', 'files/2_43763.Interview_with_the_Vampire.html', 'files/2_153747.Moby_Dick_or_the_Whale.html', 'files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html', 'files/2_4989.The_Red_Tent.html', 'files/2_37435.The_Secret_Life_of_Bees.html']


Ok we are off to parse each one of the html pages we fetched. We have provided the skeleton of the code and the code to parse the year, since it is a bit more complex...see the difference in the screenshots above. 

In [24]:
import re
yearre = r'\d{4}'
def get_year(d):
    if d.select_one("nobr.greyText"):
        return d.select_one("nobr.greyText").text.strip().split()[-1][:-1]
    else:
        thetext=d.select("div#details div.row")[1].text.strip()
        rowmatch=re.findall(yearre, thetext)
        if len(rowmatch) > 0:
            rowtext=rowmatch[0].strip()
        else:
            rowtext="NA"
        return rowtext

<div class="exercise"><b>Exercise</b></div>

Your job is to fill in the code to get the genres.

In [25]:
def get_genres(d):
    # your code here
    genres=d.select("div.elementList div.left a")
    glist=[]
    for g in genres:
        glist.append(g['href'])
    return glist

In [26]:

listofdicts=[]
for filetoread in fetched:
    print(filetoread)
    td={}
    with open(filetoread) as fd:
        datext = fd.read()
    d=BeautifulSoup(datext, 'html.parser')
    td['title']=d.select_one("meta[property='og:title']")['content']
    td['isbn']=d.select_one("meta[property='books:isbn']")['content']
    td['booktype']=d.select_one("meta[property='og:type']")['content']
    td['author']=d.select_one("meta[property='books:author']")['content']
    #td['rating']=d.select_one("span.average").text
    td['year'] = get_year(d)
    td['file']=filetoread
    glist = get_genres(d)
    td['genres']="|".join(glist)
    listofdicts.append(td)

files/1_2767052-the-hunger-games.html
files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
files/1_2657.To_Kill_a_Mockingbird.html
files/1_1885.Pride_and_Prejudice.html
files/1_41865.Twilight.html
files/2_43763.Interview_with_the_Vampire.html
files/2_153747.Moby_Dick_or_the_Whale.html
files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
files/2_4989.The_Red_Tent.html
files/2_37435.The_Secret_Life_of_Bees.html


In [27]:
listofdicts[0]

{'title': 'The Hunger Games (The Hunger Games, #1)',
 'isbn': '9780439023481',
 'booktype': 'books.book',
 'author': 'https://www.goodreads.com/author/show/153394.Suzanne_Collins',
 'year': '2008',
 'file': 'files/1_2767052-the-hunger-games.html',
 'genres': '/genres/young-adult|/genres/fiction|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action'}

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [28]:
df = pd.DataFrame.from_records(listofdicts)
df

,author,booktype,file,genres,isbn,title,year
0,https://www.goodreads.com/author/show/153394.S...,books.book,files/1_2767052-the-hunger-games.html,/genres/young-adult|/genres/fiction|/genres/sc...,9780439023481,"The Hunger Games (The Hunger Games, #1)",2008
1,https://www.goodreads.com/author/show/1077326....,books.book,files/1_2.Harry_Potter_and_the_Order_of_the_Ph...,/genres/fantasy|/genres/young-adult|/genres/fi...,9780439358071,Harry Potter and the Order of the Phoenix (Har...,2003
2,https://www.goodreads.com/author/show/1825.Har...,books.book,files/1_2657.To_Kill_a_Mockingbird.html,/genres/classics|/genres/fiction|/genres/histo...,null,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",1960
3,https://www.goodreads.com/author/show/1265.Jan...,books.book,files/1_1885.Pride_and_Prejudice.html,/genres/classics|/genres/fiction|/genres/roman...,null,Pride and Prejudice,1813
4,https://www.goodreads.com/author/show/941441.S...,books.book,files/1_41865.Twilight.html,/genres/young-adult|/genres/fantasy|/genres/ro...,9780316015844,"Twilight (Twilight, #1)",2005
5,https://www.goodreads.com/author/show/7577.Ann...,books.book,files/2_43763.Interview_with_the_Vampire.html,/genres/horror|/genres/fantasy|/genres/fiction...,9780345476876,Interview with the Vampire (The Vampire Chroni...,1976
6,https://www.goodreads.com/author/show/1624.Her...,books.book,files/2_153747.Moby_Dick_or_the_Whale.html,/genres/classics|/genres/fiction|/genres/liter...,9780142437247,"Moby-Dick, or, the Whale",1851
7,https://www.goodreads.com/author/show/1077326....,books.book,files/2_5.Harry_Potter_and_the_Prisoner_of_Azk...,/genres/fantasy|/genres/young-adult|/genres/fi...,9780439655484,Harry Potter and the Prisoner of Azkaban (Harr...,1999
8,https://www.goodreads.com/author/show/626222.A...,books.book,files/2_4989.The_Red_Tent.html,/genres/historical|/genres/historical-fiction|...,9780312353766,The Red Tent,1997
9,https://www.goodreads.com/author/show/4711.Sue...,books.book,files/2_37435.The_Secret_Life_of_Bees.html,/genres/fiction|/genres/historical|/genres/his...,9780142001745,The Secret Life of Bees,2001


In [29]:
df.to_csv("files/meta_utf8_EK.csv", index=False, header=True)